<a id=IMPORTACIÓN-CARGA-PAQUETES></a>
# IMPORTACIÓN Y CARGA DE PAQUETES

Para importar los paquetes externos debidamente, han de tenerse instalados ciertos módulos en Python. Concretamente, y para la distribución "conda", los paquetes necesarios pueden instalarse mediante:
<ul>
    <li><b style="color:#FF0000";>pickle</b>: <code>conda install -c conda-forge pickle5</code></li>
    <li><b style="color:#FF0000";>statistics</b>: <code>conda install -c conda-forge statistics</code>; es necesario para realizar ciertas operaciones de post-procesado.</li>
    <li><b style="color:#FF0000";>nptdms</b>: <code>conda install -c conda-forge nptdms</code>; es necesario para poder abrir archivos en formato TDMS, como se explica <a href=#PROCESAMIENTO-FICHEROS-TDMS>más abajo</a>.</li>
    <li><b style="color:#FF0000";>termcolor</b>: <code>conda install -c omnia termcolor</code>; necesario para poder escribir, en el terminal, texto a color. El color es empleado para dar cuenta de ciertas incidencias o fallos, así como indicar una adecuada carga de ficheros y su posterior procesamiento.</li>
    <li><b style="color:#FF0000";>dataclasses</b>: <code>conda install -c conda-forge dataclasses</code>; necesario para definir clases de datos y estructuras, de cara a desarrollar un código orientado a objetos (ver <a href=#PROCESAMIENTO-FICHEROS-TDMS>más abajo</a>).</li>
</ul>

Aunque parezca que no se hace un uso directo de todos esos módulos, se explica el porqué de su instalación a continuación. El bloque siguiente se encarga de importar los paquetes necesarios, y se clasifican en dos bloques:
<ul>
    <li>Paquetes externos:
        <ul>
            <li>numpy</li>
            <li>matplotlib</li>
            <li>statistics</li>
            <li>...</li>
        </ul></li>
    <li>Paquetes propios: son los desarrollados, expresamente, para la carga y el inicial post-procesamiento de datos provenientes de ensayos del túnel de viento. Estos paquetes <strong style="color:#FF0000";>no han de modificarse</strong>, y son los que emplean los módulos instalados aludidos anteriormente. Los paquetes propios son tres:
        <ul>
            <li>TDMSClasses: define todas las clases y métodos necesarios para crear objetos que repliquen los ficheros TDMS que provienen de los ensayos en túnel de viento.</li>
            <li>TDMSEnums: define todos las estructuras enumerativas que se emplean en TDMSClasses. Básicamente, variables que pueden tomar una serie de valores discretos.</li>
            <li>MathTools: conjunto de herramientas matemáticas necesarias para llevar a cabo ciertas operaciones de post-procesado.</li>
        </ul></li>
</ul>


In [1]:
# Paquetes externos.
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import statistics as st
import pickle
import scipy as sc
import enum
import os
import importlib

# Paquetes y módulos propios.
import own_packages.TDMS_Packages.TDMSClasses as TDMSC
import own_packages.TDMS_Packages.TDMSEnums as TDMSE
import own_packages.Math_Tools.MathTools as mt

Las siguientes clases (que heredan su estructura del objeto <code>Enum</code>, o sea, enumeraciones o estructuras de datos ordenadas) se declaran para facilitar el código posterior. Se emplean, básicamente, para:
    <ul>
    <li>clase <code>config</code>: indicar el tipo de configuración fluida al que un fichero de datos específico hace referencia.</li>
    <li>clase <code>curves</code>: indicar el tipo de curvas que contiene el fichero (sean curvas $c_{l}-c_{d}$ o curvas $c_{p}$.</li>
    <li>clase <code>mode</code>: indicar el tipo de modo que se emplea para la carga de ficheros (sea <code>"KIS"</code> para indicar que el fichero contiene medidas realizadas, únicamente, con la balanza piezoeléctrica "Kistler"; sea <code>"KIS_WR"</code> para indicar que las medidas se han realizado con la balanza y con el "wake-rake"; o sea <code>"SURFPRES"</code> para indicar que se han realizado medidas de presión superficial.</li>    
    </ul>

In [2]:
# Clase "config" para indicar el tipo de configuración fluida al que un fichero de datos hace referencia.
class config(enum.Enum):
    CLEAN = "01_CLEAN_CONFIG"
    TURB = "02_TURB_CONFIG"
    ROUGH = "03_ROUGH_CONFIG"
    REAL = "04_REAL_CONFIG"

# Clase "curves" para indicar el tipo de curvas que contiene un fichero.
class curves(enum.Enum):
    CL_CD = "CL_CD_CURVES"
    CP = "CP_CURVES"
    
# Clase "mode" para indicar el modo que se emplea para la carag de ficheros.
class mode(enum.Enum):
    KIS = "KIS"
    KIS_WR = "KIS_WR"
    SURFPRES = "SURFPRES"

<a id=PROCESAMIENTO-FICHEROS-TDMS></a>
# PROCESAMIENTO DE FICHEROS TDMS

El almacenamiento de las mediciones del túnel de viento se ha realizado mediante su volcado en ficheros TDMS, que son unos ficheros "tipo" empleados por la marca National Instruments para facilitar tanto su almacenamiento como su posterior manipulación:
<ul>
    <li>Un fichero TDMS es un fichero binario con una estructura concreta. Cada fichero TDMS representa un ensayo realizado en el túnel de viento.</li>
    <li>Existe un plug-in para poder abrir estos ficheros en formato Excel, lo cual puede ser de utilidad para ver la estructura interna de los ficheros e, incluso, para poder realizar las acciones de post-procesado pertinentes; el plug-in puede descargarse <a href=https://www.ni.com/es-es/support/downloads/tools-network/download.tdm-excel-add-in-for-microsoft-excel.html#378046>aquí</a> y, una vez instalado, es cuestión de abrir los ficheros TDMS mediante doble click.</li>
    <li>Sin embargo, el código está escrito según el paradigma OOP (<a href=https://en.wikipedia.org/wiki/Object-oriented_programming>"object oriented programming"</a>); así, lo que se hace es crear un objeto que replique el fichero TDMS en una estructura (un diccionario, concretamente) de Python.</li>
    <li>La primera parte del código consiste en cargar los ficheros TDMS, crear la estructura u objeto pertinente (el diccionario) y guardar dicho objeto en formato <a href=https://en.wikipedia.org/wiki/Serialization>serializado</a> (o, para que se entienda, un formato binario que se pueda cargar mediante un módulo de Python). Así, lo que se guarda al final del código es el objeto de Python creado (el diccionario), para que no se tenga que volver a cargar el fichero TDMS toda las veces que se vuelva a correr el código. Esto redunda en una eficiencia de carga sustancial, ya que algunos ficheros TDMS pueden tener un peso considerable.</li>
    <li>El módulo de Python que permite generar y cargar ficheros serializados es el llamado <a href=https://docs.python.org/3/library/pickle.html>Pickle</a>. Al correr el código, y tras cargar el TDMS y generar el objeto-diccionario correspondiente, éste se guardará como objeto serializado (extensión ".p") en la misma carpeta de donde proviene el fichero TDMS.</li>
    <li>Una vez que se generan los objetos serializados, no hará falta correr esta parte del código, sino que podrá saltarse la sección e ir, directamente <a href=#CARGA-DATOS-EXPERIMENTALES>a la parte correspondiente a los datos experimentales</a>. De todas formas, y si se quieren volver a cargar los ficheros TDMS por alguna razón, puede ejecutarse este bloque de código sin ningún problema. Pero, en principio, la razón por la cual se incluye esta sección es para:
        <ul>
            <li>dar una visión global de cómo funciona el código, y de hacer ver que existe una trazabilidad entre los datos en crudo y los objetos-diccionario en formato Python que están listos para ser post-procesados.</li>
            <li>ejecutarlo una primera vez (al compartir la carpeta que contiene el código y los ficheros de datos, éstos últimos están en formato TDMS (no serializados) para evitar ocupar más espacio del debido).</li>
        </ul>
    </li>
</ul>

La única sección que se ha añadido, por el momento, es el correspondiente al de las curvas $x'-c_{p}$ de la configuración limpia (hay cuatro configuraciones de flujo: limpia-turbulenta-rugosa-real) y, dentro de la misma, la que corresponde al airfoil limpio (sin elementos de rugosidad discretos empleados como modo de control de flujo). A medida que se vayan añadiendo más ficheros, como los correspondientes a las mediciones de lift y drag, será conveniente mantener los bloques de código correspondientes en subsecciones apartadas, de cara a asegurar una legibilidad del código adecuada.


## CONFIGURACIÓN LIMPIA

### AIRFOIL LIMPIO (SIN RUGOSIDADES DISCRETAS)

#### CURVAS $x'-c_{p}$

En cuanto a la denominación de los ficheros y su significado:
<ul>
    <li>Téngase en cuenta que $x'$ hace referencia al parámetro adimensional en la dirección de la cuerda del airfoil. Cada fichero se corresponde con un número de Reynolds ($\text{Re}$) concreto, y los datos incluidos en el mismo son para distintas configuraciones de ángulo de ataque ($\alpha$).
Los nombres de los ficheros de datos están puestos para identificar lo que incluyen: la forma genérica que tienen tales ficheros es:
<br></br>
<br></br>
<center><b style="color:#FF0000";>tipo-de-medición__parámetro-espacial-principal__modelo-airfoil-empleado__valor-Reynolds__rango-parámetro-espacial-principal__repetición</b></center>
<br></br>
<center>(p. ej.: Surface_pressure_angleWise_bareAirfoil_elec_Re0,8e5_AoA-20_20_2)</center>
<br></br>
<br></br>
donde:
    <ul>
        <li>El <b style="color:#FF0000";>tipo-de-medición</b> es, en este caso, <b style="color:#FF0000";>Surface_pressure</b>; o sea, indica que se han realizado medidas de presión superficial.</li> 
        <li>El <b style="color:#FF0000";>parámetro-espacial-principal</b> es, en este caso, <b style="color:#FF0000";>angleWise</b>, indicado que las distintas medidas realizadas se corresponden con diferentes configuraciones de $\alpha$.</li>
        <li>El <b style="color:#FF0000";>modelo-airfoil-empleado</b> es, en este caso, <b style="color:#FF0000";>bareAirfoil_elec</b>; la alocución <b style="color:#FF0000";>bareAirfoil</b> indica que las medidas se han realizado sobre el airfoil sin rugosidades discretas. Por su parte, el término <b>elec</b> sirve para identificar el airfoil que se ha empleado como modelo, y no reviste mayor importancia (es un parámetro que tiene relevancia para el registro interno en el túnel de viento.</li>
        <li>El <b style="color:#FF0000";>valor-Reynolds</b> toma la forma <b style="color:#FF0000";>Re#</b>, donde <b style="color:#FF0000";>#</b> indica el número de Reynolds al que se ha realizado el ensayo. Podrá tomar valores entre 0.8e5 y 1.6e5.</li>
        <li>El <b style="color:#FF0000";>rango-parámetro-espacial-principal</b> indica el rango en el que se ha variado, en este caso, el ángulo de ataque. Como todos los ensayos están realizados variando el ángulo de ataque entre $\alpha\in\left[-20, 20\right]^{\circ}$, este parámetro adquirirá la forma <b style="color:#FF0000";>AoA-20_20.</b></li>
        <li>El parámetro <b style="color:#FF0000";>repetición</b> existe para identificar los distintos ensayos que se han realizado sobre la misma configuración. Para asegurar una correcta convergencia estadística al analizar los datos, cada configuración se ha ensayado un mínimo de 3 veces. El hecho de que un fichero no tenga el parámetro de repetición indica que es el primero de los ensayos realizados sobre dicha configuración. Las dos repeticiones restantes se identifican como 2 y 3.</li>
    </ul></li>
    <li>Los bloques de código siguiente no solo realizan la carga de los ficheros TDMS, sino que también llevan a cabo parte del post-procesamiento. Concretamente:
    <ul>
        <li>Se adimensionalizan todas las variables con ciertas magnitudes de referencia. Las magnitudes de referencia que se pasan explícitamente como argumentos son la longitud de la cuerda, la longitud del "span" y el espesor máximo del airfoil correspondientes al NACA0021. De emplearse algún otro airfoil con características geométricas distintas, estas magnitudes deberían ser las de ese otro airfoil. Las magnitues de referencia de velocidad, presión, presión dinámica...se establecen automáticamente a partir de la lectura del fichero de datos. De todas formas, dichas magnitudes también pueden especificarse en caso necesario. Con todo, el diccionario-objeto de Python que se genera al final estará adimensionalizado. Dicha adimensionalización puede revertirse <i>a posteriori</i>.</li>
        <li>Se completa el análisis de incertidumbre correspondiente de acuerdo a los principios establecidos en el <a href=https://www.sciencedirect.com/science/article/pii/S0263224120304796> artículo correspondiente</a>.</li>
    </ul></li>
    <li>Tanto la lectura de los ficheros TDMS, las operaciones de post-procesado necesarias, y el volcado del objeto-diccionario serializado resultante en un fichero Pickle dependen de unas funciones ("métodos", en rigor) que se encuentran en los paquetes propios que se han cargado en la <a href=#IMPORTACIÓN-CARGA-PAQUETES>sección correspondiente</a>. Algunas puntualizaciones al respecto:
    <ul>
        <li>Los ficheros TDMS se cargan mediante el empleo de dos funciones: <code python>TdmsFileData</code> y <code python>open_list_of_files()</code>. La función <code python>TdmsFileData</code> no es una función como tal, sino la instanciación de un objeto (por eso no aparece un paréntesis <code python>()</code> al final; las funciones se distinguirán por tener que ser llamadas o invocadas de alguna manera, que es lo que los paréntesis al final del nombre de la función indican, como en el caso de <code python>open_list_of_files()</code>). Siempre que se le llama, crea un objeto correspondiente a un fichero TDMS. La descripción de los argumentos que se le pasan en el momento de la instanciación pueden verse en la documentación pertinente, haciendo <code python>TDMSC.TdmsFileData.__init__?</code>; aunque, dicho sea de paso, el primer y segundo argumentos representan, respectivamente, la ruta del fichero TDMS y el nombre que se le quiere dar al objeto instanciado. La estructura del objeto <code python>TdmsFileData</code> genérico, definido en el fichero <i>TDMSClasses.py</i>, no ha de modificarse en principio.</li>
        <li>Una vez se constituye la lista de ficheros a leer (siempre ha de ser una lista, aunque el fichero a leer sea único), cada uno de los ficheros instanciados en formato <code python>TdmsFileData</code> se lee mediante la llamada a la función <code python>open_list_of_files()</code>. Al igual que la estructura genérica <code python>TdmsFileData</code>, esta función reside en el fichero <i>TDMSClasses.py</i>, y no ha de modificarse en principio.</li>
    </ul></li>
    </ul>

In [3]:
importlib.reload(TDMSC)
# Se declara la variable 'path' para indicar la ruta (el directorio) donde se encuentran los ficheros.
path = "Exp_data/Clean_config/Bare_airfoil/cp/"
# Se declara la varaible 'list_of_files' como una lista vacía. Esta variable se compondrá de todos
# los objetos TDMS que se quieren cargar.
list_of_files = list()

#############################################################################################################
################FICHEROS DE DATOS CORRESPONDIENTES A MEDICIONES DE PRESIÓN SUPERFICIAL#################
#############################################################################################################

# Ficheros de ensayos a Re=0.8e5
list_of_files.append((TDMSC.TdmsFileData(path + "Surface_pressure_angleWise_bareAirfoil_elec_Re0,8e5_AoA-20_20/rawData_0.tdms",
                            "Bare_Re_80_e_3_m20_20_SURFPRES_HYST",
                            "",
                            TDMSE.tdmsFilereadMode.surface_pressure,
                            TDMSC.RefMagnitudes(length = 0.15, span = 0.9, thick = 0.15*0.21)
                            ), mode.SURFPRES))
list_of_files.append((TDMSC.TdmsFileData(path + "Surface_pressure_angleWise_bareAirfoil_elec_Re0,8e5_AoA-20_20_2/rawData_1.tdms",
                            "Bare_Re_80_e_3_m20_20_SURFPRES_HYST_2",
                            "",
                            TDMSE.tdmsFilereadMode.surface_pressure,
                            TDMSC.RefMagnitudes(length = 0.15, span = 0.9, thick = 0.15*0.21)
                            ), mode.SURFPRES))
list_of_files.append((TDMSC.TdmsFileData(path + "Surface_pressure_angleWise_bareAirfoil_elec_Re0,8e5_AoA-20_20_3/rawData_2.tdms",
                            "Bare_Re_80_e_3_m20_20_SURFPRES_HYST_3",
                            "",
                            TDMSE.tdmsFilereadMode.surface_pressure,
                            TDMSC.RefMagnitudes(length = 0.15, span = 0.9, thick = 0.15*0.21)
                            ), mode.SURFPRES))

# Ficheros de ensayos a Re=1e5
list_of_files.append((TDMSC.TdmsFileData(path + "Surface_pressure_angleWise_bareAirfoil_elec_Re1e5_AoA-20_20/rawData_0.tdms",
                            "Bare_Re_100_e_3_m20_20_SURFPRES_HYST",
                            "",
                            TDMSE.tdmsFilereadMode.surface_pressure,
                            TDMSC.RefMagnitudes(length = 0.15, span = 0.9, thick = 0.15*0.21)
                            ), mode.SURFPRES))
list_of_files.append((TDMSC.TdmsFileData(path + "Surface_pressure_angleWise_bareAirfoil_elec_Re1e5_AoA-20_20_2/rawData_1.tdms",
                            "BBare_Re_100_e_3_m20_20_SURFPRES_HYST_2",
                            "",
                            TDMSE.tdmsFilereadMode.surface_pressure,
                            TDMSC.RefMagnitudes(length = 0.15, span = 0.9, thick = 0.15*0.21)
                            ), mode.SURFPRES))
list_of_files.append((TDMSC.TdmsFileData(path + "Surface_pressure_angleWise_bareAirfoil_elec_Re1e5_AoA-20_20_3/rawData_2.tdms",
                            "Bare_Re_100_e_3_m20_20_SURFPRES_HYST_3",
                            "",
                            TDMSE.tdmsFilereadMode.surface_pressure,
                            TDMSC.RefMagnitudes(length = 0.15, span = 0.9, thick = 0.15*0.21)
                            ), mode.SURFPRES))

# Ficheros de ensayos a Re=1.2e5
list_of_files.append((TDMSC.TdmsFileData(path + "Surface_pressure_angleWise_bareAirfoil_elec_Re1,2e5_AoA-20_20/rawData_0.tdms",
                            "Bare_Re_120_e_3_m20_20_SURFPRES_HYST",
                            "",
                            TDMSE.tdmsFilereadMode.surface_pressure,
                            TDMSC.RefMagnitudes(length = 0.15, span = 0.9, thick = 0.15*0.21)
                            ), mode.SURFPRES))
list_of_files.append((TDMSC.TdmsFileData(path + "Surface_pressure_angleWise_bareAirfoil_elec_Re1,2e5_AoA-20_20_2/rawData_1.tdms",
                            "Bare_Re_120_e_3_m20_20_SURFPRES_HYST_2",
                            "",
                            TDMSE.tdmsFilereadMode.surface_pressure,
                            TDMSC.RefMagnitudes(length = 0.15, span = 0.9, thick = 0.15*0.21)
                            ), mode.SURFPRES))
list_of_files.append((TDMSC.TdmsFileData(path + "Surface_pressure_angleWise_bareAirfoil_elec_Re1,2e5_AoA-20_20_3/rawData_2.tdms",
                            "Bare_Re_120_e_3_m20_20_SURFPRES_HYST_3",
                            "",
                            TDMSE.tdmsFilereadMode.surface_pressure,
                            TDMSC.RefMagnitudes(length = 0.15, span = 0.9, thick = 0.15*0.21)
                            ), mode.SURFPRES))

# Ficheros de ensayos a Re=1.4e5
list_of_files.append((TDMSC.TdmsFileData(path + "Surface_pressure_angleWise_bareAirfoil_elec_Re1,4e5_AoA-20_20/rawData_0.tdms",
                            "Bare_Re_140_e_3_m20_20_SURFPRES_HYST",
                            "",
                            TDMSE.tdmsFilereadMode.surface_pressure,
                            TDMSC.RefMagnitudes(length = 0.15, span = 0.9, thick = 0.15*0.21)
                            ), mode.SURFPRES))
list_of_files.append((TDMSC.TdmsFileData(path + "Surface_pressure_angleWise_bareAirfoil_elec_Re1,4e5_AoA-20_20_2/rawData_1.tdms",
                            "Bare_Re_140_e_3_m20_20_SURFPRES_HYST_2",
                            "",
                            TDMSE.tdmsFilereadMode.surface_pressure,
                            TDMSC.RefMagnitudes(length = 0.15, span = 0.9, thick = 0.15*0.21)
                            ), mode.SURFPRES))
list_of_files.append((TDMSC.TdmsFileData(path + "Surface_pressure_angleWise_bareAirfoil_elec_Re1,4e5_AoA-20_20_3/rawData_2.tdms",
                            "Bare_Re_140_e_3_m20_20_SURFPRES_HYST_3",
                            "",
                            TDMSE.tdmsFilereadMode.surface_pressure,
                            TDMSC.RefMagnitudes(length = 0.15, span = 0.9, thick = 0.15*0.21)
                            ), mode.SURFPRES))

# Ficheros de ensayos a Re=1.6e5
list_of_files.append((TDMSC.TdmsFileData(path + "Surface_pressure_angleWise_bareAirfoil_elec_Re1,6e5_AoA-20_20/rawData_0.tdms",
                            "Bare_Re_160_e_3_m20_20_SURFPRES_HYST",
                            "",
                            TDMSE.tdmsFilereadMode.surface_pressure,
                            TDMSC.RefMagnitudes(length = 0.15, span = 0.9, thick = 0.15*0.21)
                            ), mode.SURFPRES))
list_of_files.append((TDMSC.TdmsFileData(path + "Surface_pressure_angleWise_bareAirfoil_elec_Re1,6e5_AoA-20_20_2/rawData_1.tdms",
                            "Bare_Re_160_e_3_m20_20_SURFPRES_HYST_2",
                            "",
                            TDMSE.tdmsFilereadMode.surface_pressure,
                            TDMSC.RefMagnitudes(length = 0.15, span = 0.9, thick = 0.15*0.21)
                            ), mode.SURFPRES))
list_of_files.append((TDMSC.TdmsFileData(path + "Surface_pressure_angleWise_bareAirfoil_elec_Re1,6e5_AoA-20_20_3/rawData_2.tdms",
                            "Bare_Re_160_e_3_m20_20_SURFPRES_HYST_3",
                            "",
                            TDMSE.tdmsFilereadMode.surface_pressure,
                            TDMSC.RefMagnitudes(length = 0.15, span = 0.9, thick = 0.15*0.21)
                            ), mode.SURFPRES))

# Instrucción de apertura mediante llamada al método "open_list_of_files()".
TDMSC.open_list_of_files([_[0] for _ in list_of_files], reload=True)

----------Attempting Bare_Re_80_e_3_m20_20_SURFPRES_HYST----------
Sequential runs: check correct --> Ordering: (angle) --> Numbering: (81)
Saving serialized object in Pickle file: Bare_Re_80_e_3_m20_20_SURFPRES_HYST.p
TDMS file 1 of 15 opened in surface_pressure mode; Bare_Re_80_e_3_m20_20_SURFPRES_HYST global variable has been created.
----------Attempting Bare_Re_80_e_3_m20_20_SURFPRES_HYST_2----------
Sequential runs: check correct --> Ordering: (angle) --> Numbering: (81)
Saving serialized object in Pickle file: Bare_Re_80_e_3_m20_20_SURFPRES_HYST_2.p
TDMS file 2 of 15 opened in surface_pressure mode; Bare_Re_80_e_3_m20_20_SURFPRES_HYST_2 global variable has been created.
----------Attempting Bare_Re_80_e_3_m20_20_SURFPRES_HYST_3----------
Sequential runs: check correct --> Ordering: (angle) --> Numbering: (81)
Saving serialized object in Pickle file: Bare_Re_80_e_3_m20_20_SURFPRES_HYST_3.p
TDMS file 3 of 15 opened in surface_pressure mode; Bare_Re_80_e_3_m20_20_SURFPRES_HYST_3 gl

<a id=CARGA-DATOS-EXPERIMENTALES></a>
# CARGA DE DATOS EXPERIMENTALES

Los bloques de código correspondientes a esta sección se emplean para cargar los ficheros Pickle que contienen los objetos-diccionario Python de los archivos TDMS de los ensayos realizados en el túnel de viento.

La única sección que se ha añadido, por el momento, es el correspondiente al de las curvas $x'-c_{p}$ de la configuración limpia (hay cuatro configuraciones de flujo: limpia-turbulenta-rugosa-real) y, dentro de la misma, la que corresponde al airfoil limpio (sin elementos de rugosidad discretos empleados como modo de control de flujo). A medida que se vayan añadiendo más ficheros, como los correspondientes a las mediciones de lift y drag, será conveniente mantener los bloques de código correspondientes en subsecciones apartadas, de cara a asegurar una legibilidad del código adecuada.

<a id=CONFIGURACIÓN-LIMPIA></a>
## CONFIGURACIÓN LIMPIA

<a id=CONFIGURACIÓN-LIMPIA-AIRFOIL-LIMPIO></a>
### AIRFOIL LIMPIO (SIN RUGOSIDADES DISCRETAS)

<a id=CONFIGURACIÓN-LIMPIA-AIRFOIL-LIMPIO-CURVAS-CP></a>
#### CURVAS $x'-c_{p}$

In [4]:
# Se declara la variable 'path' para indicar la ruta (el directorio) donde se encuentran los ficheros.
path = "Exp_data/Clean_config/Bare_airfoil/cp/"
# Se declara la variable 'file_groups' que incluye el conjunto de grupos de ensayos realizados sobre
# la misma configuración fluida. Es decir, los ensayos realizados a un número de Reynolds de 0.8e5
# se agrupan en una lista que, así, permite llevar a cabo operaciones de promedio sobre los datos
# de los tres ensayos y obtener las curvas medias.
files = ["Bare_Re_80_e_3_m20_20_SURFPRES_HYST",
         "Bare_Re_80_e_3_m20_20_2_SURFPRES_HYST",
         "Bare_Re_80_e_3_m20_20_3_SURFPRES_HYST",
         "Bare_Re_100_e_3_m20_20_SURFPRES_HYST",
         "Bare_Re_100_e_3_m20_20_2_SURFPRES_HYST",
         "Bare_Re_100_e_3_m20_20_3_SURFPRES_HYST",        
         "Bare_Re_120_e_3_m20_20_SURFPRES_HYST",
         "Bare_Re_120_e_3_m20_20_2_SURFPRES_HYST",
         "Bare_Re_120_e_3_m20_20_3_SURFPRES_HYST",         
         "Bare_Re_140_e_3_m20_20_SURFPRES_HYST",         
         "Bare_Re_140_e_3_m20_20_2_SURFPRES_HYST",
         "Bare_Re_140_e_3_m20_20_3_SURFPRES_HYST",       
         "Bare_Re_160_e_3_m20_20_SURFPRES_HYST",
         "Bare_Re_160_e_3_m20_20_2_SURFPRES_HYST",
         "Bare_Re_160_e_3_m20_20_3_SURFPRES_HYST"]

# Bucle sobre cada fichero Pickle para importarlo.
for file in files:
    for d in os.listdir(path):
        if file + ".p" in os.listdir(path + "/" + d):
            with open(path + "/" + d + "/" + file + ".p", 'rb') as pickle_file:
                globals()[file] = pickle.load(pickle_file)
            break
        else:
            continue
        break

In [14]:
Bare_Re_100_e_3_m20_20_SURFPRES_HYST.groups_added.surface_pressure_group.angle.angle_surface_pressure_values

[[1.1642178,
  0.4370792,
  -0.1588757,
  -0.55909157,
  -0.721755,
  -0.8203291,
  -0.8787417,
  -0.89635426,
  -0.85913837,
  -0.8640796,
  -0.8383311,
  -0.7720608,
  -0.7649720907211304,
  -0.75788337,
  -0.71776974,
  -0.69284344,
  -0.6451691,
  -0.62773854,
  -0.6014291,
  -0.57871455,
  -0.55591655,
  -0.526303,
  -0.48845148,
  -0.43473175,
  -0.37228507,
  -0.33679575,
  -0.29862368,
  -0.25584486,
  -0.23582104,
  -0.20728277],
 [1.1540931,
  0.2343809,
  -0.3756134,
  -0.7557797,
  -0.89621776,
  -0.9715731,
  -1.0137366,
  -1.0172387,
  -0.9646983,
  -0.9617554,
  -0.9236632,
  -0.84606105,
  -0.8362246155738831,
  -0.8263882,
  -0.78233474,
  -0.75447893,
  -0.70421386,
  -0.6836371,
  -0.65606177,
  -0.6373623,
  -0.61161214,
  -0.5702699,
  -0.49736103,
  -0.4476442,
  -0.39207324,
  -0.35163242,
  -0.31064683,
  -0.2631541,
  -0.24329963,
  -0.21253346],
 [1.1290823,
  0.00749776,
  -0.6121265,
  -0.9629392,
  -1.0729473,
  -1.1242398,
  -1.1494596,
  -1.1321883,
  -1.